In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Step 1: Import data from exported raw survey data

In [2]:
df = pd.read_csv("C:/Users/Owner/Desktop/AlaskaProject/Data/seatsurveymodel.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,7,8,25,26,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240,252,264,276,283,288,289,290) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Step 2: Observe and summarize data

In [3]:
# Column names overview
df.columns

Index(['participant.id_in_session', 'participant.code', 'participant.label',
       'participant._is_bot', 'participant._index_in_pages',
       'participant._max_page_index', 'participant._current_app_name',
       'participant._current_page_name', 'participant.time_started',
       'participant.visited',
       ...
       'survey.1.player.gender', 'survey.1.player.feet',
       'survey.1.player.inches', 'survey.1.player.rt',
       'survey.1.player.bizlei', 'survey.1.player.seattype',
       'survey.1.player.section', 'survey.1.player.comment',
       'survey.1.group.id_in_subsession', 'survey.1.subsession.round_number'],
      dtype='object', length=293)

Step 3: Trim width of dataframe

In [4]:
# Drop irrelevant columns
df.drop(['participant.label','participant._is_bot','participant._max_page_index',
        'participant._current_page_name','participant.visited',
        'participant.mturk_worker_id','participant.mturk_assignment_id','participant.payoff',
        'session.label','session.mturk_HITId','session.mturk_HITGroupId','session.comment', 'session.is_demo',  'session.config.real_world_currency_per_point', 'session.config.participation_fee', 'randseq.1.player.role', 'randseq.1.player.payoff', 'randseq.1.group.id_in_subsession', 'randseq.1.subsession.round_number', 'survey.1.player.id_in_group', 'survey.1.player.role', 'survey.1.player.payoff'], inplace = True, axis = 1
       )

In [5]:
# Verify removal of specified columns
df.columns

Index(['participant.id_in_session', 'participant.code',
       'participant._index_in_pages', 'participant._current_app_name',
       'participant.time_started', 'session.code',
       'randseq.1.player.id_in_group', 'randseq.1.player.page_sequence',
       'randseq.1.player.choice1', 'randseq.1.player.choice2',
       ...
       'survey.1.player.gender', 'survey.1.player.feet',
       'survey.1.player.inches', 'survey.1.player.rt',
       'survey.1.player.bizlei', 'survey.1.player.seattype',
       'survey.1.player.section', 'survey.1.player.comment',
       'survey.1.group.id_in_subsession', 'survey.1.subsession.round_number'],
      dtype='object', length=271)

Step 4: Re-label fields

In [6]:
# Rename/shorten column names
df.rename(columns=
{
"session.code": "session_code",    
"participant.id_in_session": "id_in_session",
"participant._index_in_pages": "index_in_pages",
"participant._current_app_name": "current_app_name",
"participant.time_started": "time_started",
"randseq.1.player.id_in_group": "id_in_group",
"randseq.1.player.page_sequence": "page_sequence",

"randseq.1.player.choice1": "choice1",
"randseq.1.player.distance_1": "distance_1",
"randseq.1.player.time_1": "time_1",
"randseq.1.player.price_1": "price_1",
"randseq.1.player.row_1a": "rowA_1",
"randseq.1.player.row_1b": "rowB_1",
"randseq.1.player.col_1a": "colA_1",
"randseq.1.player.col_1b": "colB_1",
"randseq.1.player.price_1a": "priceA_1",
"randseq.1.player.price_1b": "priceB_1",
"randseq.1.player.legroom_1a": "legroomA_1",
"randseq.1.player.legroom_1b": "legroomB_1",

"randseq.1.player.choice2": "choice2",
"randseq.1.player.distance_2": "distance_2",
"randseq.1.player.time_2": "time_2",
"randseq.1.player.price_2": "price_2",
"randseq.1.player.row_2a": "rowA_2",
"randseq.1.player.row_2b": "rowB_2",
"randseq.1.player.col_2a": "colA_2",
"randseq.1.player.col_2b": "colB_2",
"randseq.1.player.price_2a": "priceA_2",
"randseq.1.player.price_2b": "priceB_2",
"randseq.1.player.legroom_2a": "legroomA_2",
"randseq.1.player.legroom_2b": "legroomB_2",

"randseq.1.player.choice3": "choice3",
"randseq.1.player.distance_3": "distance_3",
"randseq.1.player.time_3": "time_3",
"randseq.1.player.price_3": "price_3",
"randseq.1.player.row_3a": "rowA_3",
"randseq.1.player.row_3b": "rowB_3",
"randseq.1.player.col_3a": "colA_3",
"randseq.1.player.col_3b": "colB_3",
"randseq.1.player.price_3a": "priceA_3",
"randseq.1.player.price_3b": "priceB_3",
"randseq.1.player.legroom_3a": "legroomA_3",
"randseq.1.player.legroom_3b": "legroomB_3",

"randseq.1.player.choice4": "choice4",
"randseq.1.player.distance_4": "distance_4",
"randseq.1.player.time_4": "time_4",
"randseq.1.player.price_4": "price_4",
"randseq.1.player.row_4a": "rowA_4",
"randseq.1.player.row_4b": "rowB_4",
"randseq.1.player.col_4a": "colA_4",
"randseq.1.player.col_4b": "colB_4",
"randseq.1.player.price_4a": "priceA_4",
"randseq.1.player.price_4b": "priceB_4",
"randseq.1.player.legroom_4a": "legroomA_4",
"randseq.1.player.legroom_4b": "legroomB_4",

"randseq.1.player.choice_5": "choice5",
"randseq.1.player.distance_5": "distance_5",
"randseq.1.player.time_5": "time_5",
"randseq.1.player.price_5": "price_5",
"randseq.1.player.row_5a": "rowA_5",
"randseq.1.player.row_5b": "rowB_5",
"randseq.1.player.col_5a": "colA_5",
"randseq.1.player.col_5b": "colB_5",
"randseq.1.player.price_5a": "priceA_5",
"randseq.1.player.price_5b": "priceB_5",
"randseq.1.player.legroom_5a": "legroomA_5",
"randseq.1.player.legroom_5b": "legroomB_5",

"randseq.1.player.choice_6": "choice6",
"randseq.1.player.distance_6": "distance_6",
"randseq.1.player.time_6": "time_6",
"randseq.1.player.price_6": "price_6",
"randseq.1.player.row_6a": "rowA_6",
"randseq.1.player.row_6b": "rowB_6",
"randseq.1.player.col_6a": "colA_6",
"randseq.1.player.col_6b": "colB_6",
"randseq.1.player.price_6a": "priceA_6",
"randseq.1.player.price_6b": "priceB_6",
"randseq.1.player.legroom_6a": "legroomA_6",
"randseq.1.player.legroom_6b": "legroomB_6",

"randseq.1.player.choice_7": "choice7",
"randseq.1.player.distance_7": "distance_7",
"randseq.1.player.time_7": "time_7",
"randseq.1.player.price_7": "price_7",
"randseq.1.player.row_7a": "rowA_7",
"randseq.1.player.row_7b": "rowB_7",
"randseq.1.player.col_7a": "colA_7",
"randseq.1.player.col_7b": "colB_7",
"randseq.1.player.price_7a": "priceA_7",
"randseq.1.player.price_7b": "priceB_7",
"randseq.1.player.legroom_7a": "legroomA_7",
"randseq.1.player.legroom_7b": "legroomB_7",

"randseq.1.player.choice_8": "choice8",
"randseq.1.player.distance_8": "distance_8",
"randseq.1.player.time_8": "time_8",
"randseq.1.player.price_8": "price_8",
"randseq.1.player.row_8a": "rowA_8",
"randseq.1.player.row_8b": "rowB_8",
"randseq.1.player.col_8a": "colA_8",
"randseq.1.player.col_8b": "colB_8",
"randseq.1.player.price_8a": "priceA_8",
"randseq.1.player.price_8b": "priceB_8",
"randseq.1.player.legroom_8a": "legroomA_8",
"randseq.1.player.legroom_8b": "legroomB_8",

"randseq.1.player.choice_9": "choice9",
"randseq.1.player.distance_9": "distance_9",
"randseq.1.player.time_9": "time_9",
"randseq.1.player.price_9": "price_9",
"randseq.1.player.row_9a": "rowA_9",
"randseq.1.player.row_9b": "rowB_9",
"randseq.1.player.col_9a": "colA_9",
"randseq.1.player.col_9b": "colB_9",
"randseq.1.player.price_9a": "priceA_9",
"randseq.1.player.price_9b": "priceB_9",
"randseq.1.player.legroom_9a": "legroomA_9",
"randseq.1.player.legroom_9b": "legroomB_9",

"randseq.1.player.choice_10": "choice10",
"randseq.1.player.distance_10": "distance_10",
"randseq.1.player.time_10": "time_10",
"randseq.1.player.price_10": "price_10",
"randseq.1.player.row_10a": "rowA_10",
"randseq.1.player.row_10b": "rowB_10",
"randseq.1.player.col_10a": "colA_10",
"randseq.1.player.col_10b": "colB_10",
"randseq.1.player.price_10a": "priceA_10",
"randseq.1.player.price_10b": "priceB_10",
"randseq.1.player.legroom_10a": "legroomA_10",
"randseq.1.player.legroom_10b": "legroomB_10",

"randseq.1.player.choice_11": "choice11",
"randseq.1.player.distance_11": "distance_11",
"randseq.1.player.time_11": "time_11",
"randseq.1.player.price_11": "price_11",
"randseq.1.player.row_11a": "rowA_11",
"randseq.1.player.row_11b": "rowB_11",
"randseq.1.player.col_11a": "colA_11",
"randseq.1.player.col_11b": "colB_11",
"randseq.1.player.price_11a": "priceA_11",
"randseq.1.player.price_11b": "priceB_11",
"randseq.1.player.legroom_11a": "legroomA_11",
"randseq.1.player.legroom_11b": "legroomB_11",

"randseq.1.player.choice_12": "choice12",
"randseq.1.player.distance_12": "distance_12",
"randseq.1.player.time_12": "time_12",
"randseq.1.player.price_12": "price_12",
"randseq.1.player.row_12a": "rowA_12",
"randseq.1.player.row_12b": "rowB_12",
"randseq.1.player.col_12a": "colA_12",
"randseq.1.player.col_12b": "colB_12",
"randseq.1.player.price_12a": "priceA_12",
"randseq.1.player.price_12b": "priceB_12",
"randseq.1.player.legroom_12a": "legroomA_12",
"randseq.1.player.legroom_12b": "legroomB_12",

"randseq.1.player.choice_13": "choice13",
"randseq.1.player.distance_13": "distance_13",
"randseq.1.player.time_13": "time_13",
"randseq.1.player.price_13": "price_13",
"randseq.1.player.row_13a": "rowA_13",
"randseq.1.player.row_13b": "rowB_13",
"randseq.1.player.col_13a": "colA_13",
"randseq.1.player.col_13b": "colB_13",
"randseq.1.player.price_13a": "priceA_13",
"randseq.1.player.price_13b": "priceB_13",
"randseq.1.player.legroom_13a": "legroomA_13",
"randseq.1.player.legroom_13b": "legroomB_13",

"randseq.1.player.choice_14": "choice14",
"randseq.1.player.distance_14": "distance_14",
"randseq.1.player.time_14": "time_14",
"randseq.1.player.price_14": "price_14",
"randseq.1.player.row_14a": "rowA_14",
"randseq.1.player.row_14b": "rowB_14",
"randseq.1.player.col_14a": "colA_14",
"randseq.1.player.col_14b": "colB_14",
"randseq.1.player.price_14a": "priceA_14",
"randseq.1.player.price_14b": "priceB_14",
"randseq.1.player.legroom_14a": "legroomA_14",
"randseq.1.player.legroom_14b": "legroomB_14",

"randseq.1.player.choice_15": "choice15",
"randseq.1.player.distance_15": "distance_15",
"randseq.1.player.time_15": "time_15",
"randseq.1.player.price_15": "price_15",
"randseq.1.player.row_15a": "rowA_15",
"randseq.1.player.row_15b": "rowB_15",
"randseq.1.player.col_15a": "colA_15",
"randseq.1.player.col_15b": "colB_15",
"randseq.1.player.price_15a": "priceA_15",
"randseq.1.player.price_15b": "priceB_15",
"randseq.1.player.legroom_15a": "legroomA_15",
"randseq.1.player.legroom_15b": "legroomB_15",

"randseq.1.player.choice_16": "choice16",
"randseq.1.player.distance_16": "distance_16",
"randseq.1.player.time_16": "time_16",
"randseq.1.player.price_16": "price_16",
"randseq.1.player.row_16a": "rowA_16",
"randseq.1.player.row_16b": "rowB_16",
"randseq.1.player.col_16a": "colA_16",
"randseq.1.player.col_16b": "colB_16",
"randseq.1.player.price_16a": "priceA_16",
"randseq.1.player.price_16b": "priceB_16",
"randseq.1.player.legroom_16a": "legroomA_16",
"randseq.1.player.legroom_16b": "legroomB_16",

"randseq.1.player.choice_17": "choice17",
"randseq.1.player.distance_17": "distance_17",
"randseq.1.player.time_17": "time_17",
"randseq.1.player.price_17": "price_17",
"randseq.1.player.row_17a": "rowA_17",
"randseq.1.player.row_17b": "rowB_17",
"randseq.1.player.col_17a": "colA_17",
"randseq.1.player.col_17b": "colB_17",
"randseq.1.player.price_17a": "priceA_17",
"randseq.1.player.price_17b": "priceB_17",
"randseq.1.player.legroom_17a": "legroomA_17",
"randseq.1.player.legroom_17b": "legroomB_17",

"randseq.1.player.choice_18": "choice18",
"randseq.1.player.distance_18": "distance_18",
"randseq.1.player.time_18": "time_18",
"randseq.1.player.price_18": "price_18",
"randseq.1.player.row_18a": "rowA_18",
"randseq.1.player.row_18b": "rowB_18",
"randseq.1.player.col_18a": "colA_18",
"randseq.1.player.col_18b": "colB_18",
"randseq.1.player.price_18a": "priceA_18",
"randseq.1.player.price_18b": "priceB_18",
"randseq.1.player.legroom_18a": "legroomA_18",
"randseq.1.player.legroom_18b": "legroomB_18",

"randseq.1.player.choice_19": "choice19",
"randseq.1.player.distance_19": "distance_19",
"randseq.1.player.time_19": "time_19",
"randseq.1.player.price_19": "price_19",
"randseq.1.player.row_19a": "rowA_19",
"randseq.1.player.row_19b": "rowB_19",
"randseq.1.player.col_19a": "colA_19",
"randseq.1.player.col_19b": "colB_19",
"randseq.1.player.price_19a": "priceA_19",
"randseq.1.player.price_19b": "priceB_19",
"randseq.1.player.legroom_19a": "legroomA_19",
"randseq.1.player.legroom_19b": "legroomB_19",

"randseq.1.player.choice_20": "choice20",
"randseq.1.player.distance_20": "distance_20",
"randseq.1.player.time_20": "time_20",
"randseq.1.player.price_20": "price_20",
"randseq.1.player.row_20a": "rowA_20",
"randseq.1.player.row_20b": "rowB_20",
"randseq.1.player.col_20a": "colA_20",
"randseq.1.player.col_20b": "colB_20",
"randseq.1.player.price_20a": "priceA_20",
"randseq.1.player.price_20b": "priceB_20",
"randseq.1.player.legroom_20a": "legroomA_20",
"randseq.1.player.legroom_20b": "legroomB_20",

"randseq.1.player.choice_21": "choice21",
"randseq.1.player.distance_21": "distance_21",
"randseq.1.player.time_21": "time_21",
"randseq.1.player.price_21": "price_21",
"randseq.1.player.row_21a": "rowA_21",
"randseq.1.player.row_21b": "rowB_21",
"randseq.1.player.col_21a": "colA_21",
"randseq.1.player.col_21b": "colB_21",
"randseq.1.player.price_21a": "priceA_21",
"randseq.1.player.price_21b": "priceB_21",
"randseq.1.player.legroom_21a": "legroomA_21",
"randseq.1.player.legroom_21b": "legroomB_21",

"survey.1.player.age": "age",
"survey.1.player.gender": "gender",
"survey.1.player.feet": "height_feet",
"survey.1.player.inches": "height_inches",
"survey.1.player.rt": "annual_trip_freq",
"survey.1.player.bizlei": "percentage_biz",
"survey.1.player.seattype": "seat_type_preference",
"survey.1.player.section": "section_preference",
"survey.1.player.comment": "comment"

}, inplace=True)

Step 5: Remove incomplete survey responses

In [7]:
# Filter to keep only completed survey rows
filtered_df = df.query('current_app_name == "survey" and session_code == "s6qkz77s" or current_app_name == "survey"  and session_code =="ei2phjog"')
filtered_df.info()
#filtered_df.describe()
#filtered_df.head()
#len(filtered_df.index)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 24 to 5091
Columns: 271 entries, id_in_session to survey.1.subsession.round_number
dtypes: float64(26), int64(152), object(93)
memory usage: 605.6+ KB


Now that the data is trimmed into a manageable size, we will transform/clean it to be used for modeling.
Each row in our current dataframe corresponds to a participant with over 100 columns of fields that span 21 questions. With the intent of using Logit Regression analysis for modeling the binary probability of choosing seat A or seat B (21 pairs), we decided to break each participant row into 21 rows with each row corresponding to a unique participant-question combination.
To do this, we utilized the Pandas wide_to_long function to transform the dataframe (see code below):

In [8]:
#Create 21 rows for each person
df_clean = pd.wide_to_long(filtered_df, ['choice','price_','distance_','time_','legroomA_','legroomB_','priceA_','priceB_',
                                         'colA_','colB_','rowA_','rowB_'], i = 'participant.code', 
                                           j = 'question').sort_values(['participant.code','question'])

In [9]:
#Create data frame
##df_clean = df_clean.reset_index()
##df_clean

Furthermore, to eliminate skewness/bias in our analysis, we decided to scale the continuous variables (i.e. legroom, price, height, age, distance).

[To confirm with Alaska]
There is uncertainty regarding whether to scale the incremental price versus scale the total price as it is a possibility that the survey respondent did not factor the base price into their decision-making. Another consideration is to classify age as a categorical variable instead of a continuous variable pending the age distribution of the survey respondents.

In [10]:
#Binary Variable on whether they chose Seat A
df_clean["choice_binary"] = np.where(df_clean["choice"] == 'Seat A', 1, 0)

In [11]:
#Adding Seat Class
df_clean["seatA_class"] = np.where(df_clean["rowA_"].between(1,4), "First", 
                            np.where(df_clean["rowA_"].between(5,8), "Premium", "Economy"))
df_clean["seatB_class"] = np.where(df_clean["rowB_"].between(1,4), "First", 
                            np.where(df_clean["rowB_"].between(5,8), "Premium", "Economy"))

In [12]:
#Turn Columns into Window/Middle/Aisle
df_clean["colA_categorical"] = np.where((df_clean["colA_"] == 'A') | (df_clean["colA_"] == 'F') | ((df_clean["colA_"] == 'D') & (df_clean["rowA_"] < 5)), "Window",
                                np.where(((df_clean["colA_"] == 'B') & (df_clean["rowA_"] > 4)) | (df_clean["colA_"] == 'E'), "Middle", "Aisle"))

df_clean["colB_categorical"] = np.where((df_clean["colB_"] == 'A') | (df_clean["colB_"] == 'F') | ((df_clean["colB_"] == 'D') & (df_clean["rowB_"] < 5)), "Window",
                                np.where(((df_clean["colB_"] == 'B') & (df_clean["rowB_"] > 4)) | (df_clean["colB_"] == 'E'), "Middle", "Aisle"))

In [13]:
#Adding Left/Right in Plane
df_clean["plane_sideA"] = np.where((df_clean["colA_"] == 'A') | (df_clean["colA_"] == 'B') | ((df_clean["colA_"] == 'C') & (df_clean["rowA_"] > 4)), 
                                  "Left", "Right")
df_clean["plane_sideB"] = np.where((df_clean["colB_"] == 'A') | (df_clean["colB_"] == 'B') | ((df_clean["colB_"] == 'C') & (df_clean["rowB_"] > 4)), 
                                  "Left", "Right")

In [14]:
#Adding Distance to Bathroom Variable
df_clean["bathroom_distA"] = np.where(df_clean["rowA_"] == 27, 0, (26 - df_clean["rowA_"]))
df_clean["bathroom_distB"] = np.where(df_clean["rowB_"] == 27, 0, (26 - df_clean["rowB_"]))
df_clean["bathroom_dist_diff"] = df_clean["bathroom_distA"] - df_clean["bathroom_distB"]

In [15]:
#Adding Price Difference Variables
df_clean["price_diff"] = df_clean["priceA_"] - df_clean["priceB_"]
df_clean["price_diff_rel"] = (df_clean["priceA_"] - df_clean["priceB_"]) / df_clean["price_"]

In [16]:
#Adding Legroom Difference Variable
df_clean["legroom_diff"] = df_clean["legroomA_"] - df_clean["legroomB_"]


In [17]:
#Adding Height Variable
df_clean["height"] = df_clean["height_feet"]*12.0 + df_clean["height_inches"]


In [20]:
df_clean.drop(["index_in_pages", "survey.1.group.id_in_subsession", "time_started", "current_app_name", "id_in_group",
                  "id_in_session", "page_sequence", "survey.1.subsession.round_number", "session_code"],
                      inplace = True, axis = 1)

KeyError: "['index_in_pages' 'survey.1.group.id_in_subsession' 'time_started'\n 'current_app_name' 'id_in_group' 'id_in_session' 'page_sequence'\n 'survey.1.subsession.round_number' 'session_code'] not found in axis"

In [21]:
df_clean.to_csv('C:/Users/Owner/Desktop/AlaskaProject/Data/seatsurvey_clean3.csv', header=True)

In [22]:
#Order Columns
df_clean = df_clean[['choice_binary','rowA_','rowB_','colA_','colB_','colA_categorical','colB_categorical',
                     'seatA_class','seatB_class','plane_sideA', 'plane_sideB','priceA_','priceB_','price_diff','price_diff_rel',
                     'legroomA_','legroomB_', 'legroom_diff','bathroom_distA','bathroom_distB','bathroom_dist_diff','time_',
                     'distance_','price_','choice',
                     'seat_type_preference', 'gender', 'height_feet','annual_trip_freq', 
                     'section_preference', 'height_inches', 'comment', 'age', 'percentage_biz']]


In [23]:
df_clean.head()

choice_binary  rowA_  rowB_ colA_ colB_  \
participant.code question                                            
05etwf7w         1                     1     13     27     E     E   
                 2                     0     10     10     E     F   
                 3                     1     13     13     A     C   
                 4                     1     14      8     F     E   
                 5                     1      5     21     F     C   

                          colA_categorical colB_categorical seatA_class  \
participant.code question                                                 
05etwf7w         1                  Middle           Middle     Economy   
                 2                  Middle           Window     Economy   
                 3                  Window            Aisle     Economy   
                 4                  Window           Middle     Economy   
                 5                  Window            Aisle     Premium   

                          seatB_class plane_sideA  ...  choice  \
participant.code question                          ...           
05etwf7w         1            Economy       Right  ...  Seat A   
                 2            Economy       Right  ...  Seat B   
                 3            Economy        Left  ...  Seat A   
                 4            Premium       Right  ...  Seat A   
                 5            Economy       Right  ...  Seat A   

                           seat_type_preference  gender  height_feet  \
participant.code question                                              
05etwf7w         1                       Window  Female          5.0   
                 2                       Window  Female          5.0   
                 3                       Window  Female          5.0   
                 4                       Window  Female          5.0   
                 5                       Window  Female          5.0   

                           annual_trip_freq  section_preference  \
participant.code question                                         
05etwf7w         1                      3.0             Economy   
                 2                      3.0             Economy   
                 3                      3.0             Economy   
                 4                      3.0             Economy   
                 5                      3.0             Economy   

                           height_inches  comment   age  percentage_biz  
participant.code question                                                
05etwf7w         1                   2.0      NaN  42.0             0.0  
                 2                   2.0      NaN  42.0             0.0  
                 3                   2.0      NaN  42.0             0.0  
                 4                   2.0      NaN  42.0             0.0  
                 5                   2.0      NaN  42.0             0.0  

[5 rows x 34 columns]

In [24]:
df_clean['percentage_biz'].describe()

count    5985.000000
mean       22.575439
std        33.529310
min         0.000000
25%         0.000000
50%         1.000000
75%        50.000000
max       100.000000
Name: percentage_biz, dtype: float64

In [25]:
bizcutlabels = ['leisure','business']
bizcutbins = [-1,50,101]

df_clean['flyertype']=pd.cut(df_clean['percentage_biz'], bins=bizcutbins, labels=bizcutlabels)
df_clean['flyertype'].value_counts()

leisure     4872
business    1113
Name: flyertype, dtype: int64

In [26]:
df_clean['annual_trip_freq'].describe()

count    5985.000000
mean        7.045614
std         9.324284
min         0.000000
25%         2.000000
50%         4.000000
75%         8.000000
max        60.000000
Name: annual_trip_freq, dtype: float64

In [27]:
tripcutlabels = ['nonflyer','typical','frequent']
tripcutbins = [-1,0,4,100]

df_clean['flyerfreq']=pd.cut(df_clean['annual_trip_freq'], bins=tripcutbins, labels=tripcutlabels)
df_clean['flyerfreq'].value_counts()

typical     3087
frequent    2730
nonflyer     168
Name: flyerfreq, dtype: int64

Create dummy variables for logistic regressions. 

In [28]:
df_clean_dummies = pd.get_dummies(df_clean, columns = ['colA_categorical','colB_categorical','seatA_class','seatB_class',
                                                      'plane_sideA','plane_sideB','time_','flyerfreq','flyertype'] )
df_clean_dummies

choice_binary  rowA_  rowB_ colA_ colB_  priceA_  \
participant.code question                                                     
05etwf7w         1                     1     13     27     E     E       19   
                 2                     0     10     10     E     F        3   
                 3                     1     13     13     A     C        1   
                 4                     1     14      8     F     E       17   
                 5                     1      5     21     F     C       53   
...                                  ...    ...    ...   ...   ...      ...   
zoi82c47         17                    1     26      9     F     A       19   
                 18                    0     13     17     D     C        0   
                 19                    1      8      6     A     E       39   
                 20                    1     11     16     A     F        2   
                 21                    1     10     23     A     B        1   

                           priceB_  price_diff  price_diff_rel  legroomA_  \
participant.code question                                                   
05etwf7w         1               0          19        0.078189          0   
                 2               7          -4       -0.018957          1   
                 3               4          -3       -0.013274          0   
                 4              79         -62       -0.300971          3   
                 5               5          48        0.338028          3   
...                            ...         ...             ...        ...   
zoi82c47         17              0          19        0.065744          0   
                 18              0           0        0.000000          1   
                 19             48          -9       -0.032258          2   
                 20              0           2        0.008130          1   
                 21              0           1        0.003717          1   

                           ...  plane_sideA_Right  plane_sideB_Left  \
participant.code question  ...                                        
05etwf7w         1         ...                  1                 0   
                 2         ...                  1                 0   
                 3         ...                  0                 1   
                 4         ...                  1                 0   
                 5         ...                  1                 1   
...                        ...                ...               ...   
zoi82c47         17        ...                  1                 1   
                 18        ...                  1                 1   
                 19        ...                  0                 0   
                 20        ...                  0                 0   
                 21        ...                  0                 1   

                           plane_sideB_Right  time__Daytime  time__Red-eye  \
participant.code question                                                    
05etwf7w         1                         1              1              0   
                 2                         1              0              1   
                 3                         0              1              0   
                 4                         1              0              1   
                 5                         0              1              0   
...                                      ...            ...            ...   
zoi82c47         17                        0              1              0   
                 18                        0              1              0   
                 19                        1              1              0   
                 20                        1              1              0   
                 21                        0              1              0   

                           flyerfreq_nonflyer  flyerfreq_t

In [29]:
df_clean_dummies.columns

Index(['choice_binary', 'rowA_', 'rowB_', 'colA_', 'colB_', 'priceA_',
       'priceB_', 'price_diff', 'price_diff_rel', 'legroomA_', 'legroomB_',
       'legroom_diff', 'bathroom_distA', 'bathroom_distB',
       'bathroom_dist_diff', 'distance_', 'price_', 'choice',
       'seat_type_preference', 'gender', 'height_feet', 'annual_trip_freq',
       'section_preference', 'height_inches', 'comment', 'age',
       'percentage_biz', 'colA_categorical_Aisle', 'colA_categorical_Middle',
       'colA_categorical_Window', 'colB_categorical_Aisle',
       'colB_categorical_Middle', 'colB_categorical_Window',
       'seatA_class_Economy', 'seatA_class_First', 'seatA_class_Premium',
       'seatB_class_Economy', 'seatB_class_First', 'seatB_class_Premium',
       'plane_sideA_Left', 'plane_sideA_Right', 'plane_sideB_Left',
       'plane_sideB_Right', 'time__Daytime', 'time__Red-eye',
       'flyerfreq_nonflyer', 'flyerfreq_typical', 'flyerfreq_frequent',
       'flyertype_leisure', 'flyertype_busi

In [30]:
df_clean_dummies.drop(['choice','seat_type_preference','gender','height_feet', 'annual_trip_freq', 
                                  'section_preference', 'height_inches', 'comment', 'age', 'percentage_biz',
                                  'colA_','colB_','priceA_','priceB_','legroomA_','legroomB_','bathroom_distA','bathroom_distB'],
                                  inplace = True, axis = 1)


In [31]:
df_clean_dummies.to_csv('C:/Users/Owner/Desktop/AlaskaProject/Data/modeldata.csv', header=True)